# Guided Project on Designing and Creating a Database
Feb 5, 2017 | Soham Sen | Dataquest Data Science Course

## Imports, Data Load, Helper Functions

In [124]:
import pandas as pd
import sqlite3
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

Note the game_log.csv file is > 130MB so I'm not putting it up on git hub. It's downloadable from [here](http://www.retrosheet.org/gamelogs/index.html)

In [125]:
files=['game_log.csv', 'park_codes.csv', 'person_codes.csv', 'team_codes.csv']
data = {}

for each in files: 
    title = each.split('.')[0]
    data[title] = pd.read_csv(each) 


/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [126]:
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        #conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

In [127]:
def dropTable(name):
    q = "DROP TABLE IF EXISTS "+name+";"
    run_command(q)

## Getting to Know the Data

In [128]:
data['game_log'].columns.tolist()

['date',
 'number_of_game',
 'day_of_week',
 'v_name',
 'v_league',
 'v_game_number',
 'h_name',
 'h_league',
 'h_game_number',
 'v_score',
 'h_score',
 'length_outs',
 'day_night',
 'completion',
 'forefeit',
 'protest',
 'park_id',
 'attendance',
 'length_minutes',
 'v_line_score',
 'h_line_score',
 'v_at_bats',
 'v_hits',
 'v_doubles',
 'v_triples',
 'v_homeruns',
 'v_rbi',
 'v_sacrifice_hits',
 'v_sacrifice_flies',
 'v_hit_by_pitch',
 'v_walks',
 'v_intentional_walks',
 'v_strikeouts',
 'v_stolen_bases',
 'v_caught_stealing',
 'v_grounded_into_double',
 'v_first_catcher_interference',
 'v_left_on_base',
 'v_pitchers_used',
 'v_individual_earned_runs',
 'v_team_earned_runs',
 'v_wild_pitches',
 'v_balks',
 'v_putouts',
 'v_assists',
 'v_errors',
 'v_passed_balls',
 'v_double_plays',
 'v_triple_plays',
 'h_at_bats',
 'h_hits',
 'h_doubles',
 'h_triples',
 'h_homeruns',
 'h_rbi',
 'h_sacrifice_hits',
 'h_sacrifice_flies',
 'h_hit_by_pitch',
 'h_walks',
 'h_intentional_walks',
 'h_stri

In [129]:
data['game_log'].shape

(171907, 161)

In [130]:
data['game_log'].head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

Lots of details about the particular game for both the visiting and home teams. The park, location, the game outcome, key game statitics, who play, who was an umpire, etc. 

The park, visiting team and players are noted by a code. So those are contained in the helper files. 

In [131]:
data['park_codes'].shape

(252, 9)

In [132]:
data['park_codes'].sample(5)

,park_id,name,aka,city,state,start,end,league,notes
48,CIN05,League Park II,NaN,Cincinnati,OH,04/20/1894,10/02/1901,NL,NaN
145,NYC06,Metropolitan Park,NaN,New York,NY,05/13/1884,08/23/1884,AA,NaN
201,SFO02,Candlestick Park,3Com Park,San Francisco,CA,04/12/1960,09/30/1999,NL,NaN
217,SYR01,Star Park I,Newell Park,Syracuse,NY,05/28/1879,09/10/1879,NL,NaN
38,CHI07,South Side Park II,NaN,Chicago,IL,04/30/1890,09/27/1893,NaN,CHP:1890;CHN:1891(Tu/Th/Sa);1892-93


In [133]:
print(data['person_codes'].shape)
data['person_codes'].sample(5)

(20494, 7)


,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
18231,taylc103,Taylor,Chink,04/18/1925,NaN,NaN,NaN
11527,martm101,Martin,Morrie,04/25/1949,NaN,NaN,NaN
17724,stepj002,Stephens,Jackson,07/01/2017,NaN,NaN,NaN
17765,stewc001,Stewart,Chris,09/06/2006,NaN,NaN,NaN
3040,cey-r001,Cey,Ron,09/03/1971,NaN,NaN,NaN


In [134]:
print(data['team_codes'].shape)
data['team_codes'].sample(5)

(150, 8)


,team_id,league,start,end,city,nickname,franch_id,seq
69,LAA,AL,1961,1964,Los Angeles,Angels,LAA,1
90,PH1,NaN,1871,1875,Philadelphia,Athletics,PH1,1
7,BL4,NaN,1873,1873,Baltimore,Marylands,BL4,1
128,TL2,AA,1890,1890,Toledo,Maumees,TL2,1
75,MID,NaN,1872,1872,Middletown,Mansfields,MID,1


## Load Data into SQLite

#RETAINS STATE SO NO NEED TO RUN AGAIN
```
for index, val in data.items():
    print(index)
    with sqlite3.connect('mlb.db') as conn:
        data[index].to_sql(index, conn, index=False)
```

In [135]:

def show_tables():
    qry = "SELECT name, type FROM sqlite_master WHERE type IN ('table', 'view');"
    return run_query(qry)

show_tables()

,name,type
0,game_log,table
1,team_codes,table
2,person_codes,table
3,park_codes,table
4,park,table
5,league,table
6,team_appearance,table
7,person_appearance,table
8,person,table
9,appearance_type,table


In [136]:
q = '''
select 
    date, 
    h_name, 
    number_of_game, 
    date || h_name || number_of_game id
from game_log
limit 5;
'''

run_query(q)

,date,h_name,number_of_game,id
0,18710504,FW1,0,18710504FW10
1,18710505,WS3,0,18710505WS30
2,18710506,RC1,0,18710506RC10
3,18710508,CH1,0,18710508CH10
4,18710509,TRO,0,18710509TRO0


```
q = '''
ALTER TABLE game_log
ADD COLUMN IF NOT EXISTS game_id text;
'''

run_command(q)
```

In [137]:
q = '''
UPDATE game_log
SET game_id = h_name || date || number_of_game
'''

run_command(q)

In [138]:
q = '''
select 
    game_id,
    date, 
    h_name, 
    number_of_game 
from game_log
limit 5;
'''

run_query(q)

,game_id,date,h_name,number_of_game
0,FW1187105040,18710504,FW1,0
1,WS3187105050,18710505,WS3,0
2,RC1187105060,18710506,RC1,0
3,CH1187105080,18710508,CH1,0
4,TRO187105090,18710509,TRO,0


player names, 
_umpire_names
v_league
h_league
_manager_Name
_player_n_name, def_pos

## Normalization
### Create Tables without Foreign Keys
#### Person Table

In [139]:
dropTable('person')

person_table_query = '''
CREATE TABLE IF NOT EXISTS person (
  person_id TEXT PRIMARY KEY,
  first_name TEXT,
  last_name TEXT
);
'''

person_table_query_fill = '''
INSERT OR IGNORE INTO person
SELECT id, first, last FROM person_codes;
'''

run_command(person_table_query)

run_command(person_table_query_fill)

q = 'select * from person limit 10;'
run_query(q)


,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad
5,abadf001,Fernando,Abad
6,abadj101,John,Abadie
7,abbae101,Ed,Abbaticchio
8,abbeb101,Bert,Abbey
9,abbec101,Charlie,Abbey


#### Park Table

In [140]:
park_table_query = '''
CREATE TABLE IF NOT EXISTS park (
  park_id TEXT PRIMARY KEY,
  name TEXT,
  nickname TEXT, 
  city TEXT,
  state TEXT,
  notes TEXT
);
'''

park_table_query_fill = '''
INSERT OR IGNORE INTO park
SELECT park_id, name, aka, city, state, notes FROM park_codes;
'''

run_command(park_table_query)

run_command(park_table_query_fill)

q = 'select * from park limit 10;'
run_query(q)


,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None
5,ATL01,Atlanta-Fulton County Stadium,None,Atlanta,GA,None
6,ATL02,Turner Field,None,Atlanta,GA,None
7,ATL03,Suntrust Park,None,Atlanta,GA,None
8,BAL01,Madison Avenue Grounds,None,Baltimore,MD,WS3
9,BAL02,Newington Park,None,Baltimore,MD,BL1:1872-74; BL4:1873; BL2: 1882


#### League Table

In [141]:
league_list = data['team_codes'].league.value_counts().index
league_list

for each in league_list:
    print(each)

NL
AL
AA
UA
FL
PL


In [142]:
league_table_query = '''
CREATE TABLE IF NOT EXISTS league (
  league_id TEXT PRIMARY KEY,
  name TEXT
);
'''

run_command(league_table_query)

run_command("INSERT OR IGNORE INTO league VALUES ('NL','National League');")
run_command("INSERT OR IGNORE INTO league VALUES ('AL','American League');")
run_command("INSERT OR IGNORE INTO league VALUES ('AA','Minor League Double A');")
run_command("INSERT OR IGNORE INTO league VALUES ('UA','Union Association');")
run_command("INSERT OR IGNORE INTO league VALUES ('FL','Florida Winter League');")
run_command("INSERT OR IGNORE INTO league VALUES ('PL','Players League');")



'''old
i = 1
for each in league_list:
    i2 = str(i)
    league_table_query_fill = "INSERT OR IGNORE INTO league VALUES ("+i2+",'"+each+"');"
    print(league_table_query_fill)    
    run_command(league_table_query_fill)
    i+=1
'''

q = 'select * from league limit 10;'
run_query(q)


,league_id,name
0,NL,National League
1,AL,American League
2,AA,Minor League Double A
3,UA,Union Association
4,FL,Florida Winter League
5,PL,Players League


#### Appearance Table

In [143]:
data['appearance_type']=pd.read_csv('appearance_type.csv')

In [144]:
data['appearance_type'].head()

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [145]:
q = 'DROP TABLE IF EXISTS appearance_type;'
run_command(q)

conn = sqlite3.connect('mlb.db')
data['appearance_type'].to_sql('appearance_type', conn, index=False)

q = 'SELECT * FROM appearance_type LIMIT 10;'
run_query(q)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


### Team Table

In [146]:
q = 'DROP TABLE IF EXISTS team'
run_command(q)

team_table_query = '''
CREATE TABLE IF NOT EXISTS team (
  team_id TEXT PRIMARY KEY,
  league_id TEXT, 
  city TEXT, 
  nickname TEXT,
  franch_id TEXT,
  FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''

run_command(team_table_query)

team_table_query_fill = '''
INSERT OR IGNORE INTO team
SELECT team_id, league, city, nickname, franch_id FROM team_codes;
'''
run_command(team_table_query_fill)

q='SELECT * FROM team LIMIT 5;'
run_query(q)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [147]:
dropTable('game')

q = '''
CREATE TABLE IF NOT EXISTS game (
  game_id TEXT PRIMARY KEY,
  date TEXT, 
  number_of_game INTEGER, 
  park_id TEXT,
  length_outs TEXT,
  day BOOLEAN,
  completion TEXT,
  forefeit TEXT,
  protest TEXT,
  attendance REAL,
  length_minutes REAL,
  additional_info TEXT,
  acquisition_info TEXT,
  FOREIGN KEY (park_id) REFERENCES park(park_id)
);
'''

run_command(q)

q = '''
INSERT OR IGNORE INTO game
SELECT 
    game_id, 
    date, 
    number_of_game,
    park_id,
    length_outs, 
    day_night,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
'''
run_command(q)

q='''
SELECT *
FROM game LIMIT 5;
'''
run_query(q)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54.0,D,None,None,None,200.0,120.0,None,Y
1,WS3187105050,18710505,0,WAS01,54.0,D,None,None,None,5000.0,145.0,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54.0,D,None,None,None,1000.0,140.0,None,Y
3,CH1187105080,18710508,0,CHI01,54.0,D,None,None,None,5000.0,150.0,None,Y
4,TRO187105090,18710509,0,TRO01,54.0,D,None,None,None,3250.0,145.0,HTBF,Y


In [148]:
show_tables()

,name,type
0,game_log,table
1,team_codes,table
2,person_codes,table
3,park_codes,table
4,park,table
5,league,table
6,team_appearance,table
7,person_appearance,table
8,person,table
9,appearance_type,table


## Complex Tables
Complex in the sense that they have compound primary keys or multiple references to other tables. 

### Team Appearance Table

In [149]:
q = '''
SELECT sql FROM sqlite_master
WHERE name = "game_log"
  AND type = "table";
'''
r = run_query(q)

from IPython.core.display import HTML
HTML(r.iloc[0]['sql'])

In [150]:
q = 'PRAGMA foreign_keys = OFF;'
run_command(q)

In [151]:
st = '"v_line_score" TEXT, "h_line_score" TEXT, "v_at_bats" REAL, "v_hits" REAL, "v_doubles" REAL, "v_triples" REAL, "v_homeruns" REAL, "v_rbi" REAL, "v_sacrifice_hits" REAL, "v_sacrifice_flies" REAL, "v_hit_by_pitch" REAL, "v_walks" REAL, "v_intentional_walks" REAL, "v_strikeouts" REAL, "v_stolen_bases" REAL, "v_caught_stealing" REAL, "v_grounded_into_double" REAL, "v_first_catcher_interference" REAL, "v_left_on_base" REAL, "v_pitchers_used" REAL, "v_individual_earned_runs" REAL, "v_team_earned_runs" REAL, "v_wild_pitches" REAL, "v_balks" REAL, "v_putouts" REAL, "v_assists" REAL, "v_errors" REAL, "v_passed_balls" REAL, "v_double_plays" REAL, "v_triple_plays" REAL'
st = st.replace('\"','')
st = st.replace('v_', '')
print(st)

line_score TEXT, h_line_score TEXT, at_bats REAL, hits REAL, doubles REAL, triples REAL, homeruns REAL, rbi REAL, sacrifice_hits REAL, sacrifice_flies REAL, hit_by_pitch REAL, walks REAL, intentional_walks REAL, strikeouts REAL, stolen_bases REAL, caught_stealing REAL, grounded_into_double REAL, first_catcher_interference REAL, left_on_base REAL, pitchers_used REAL, individual_earned_runs REAL, team_earned_runs REAL, wild_pitches REAL, balks REAL, putouts REAL, assists REAL, errors REAL, passed_balls REAL, double_plays REAL, triple_plays REAL


In [152]:
team_appearance_table_query = '''
CREATE TABLE IF NOT EXISTS team_appearance (
  team_id TEXT,
  game_id TEXT,
  home TEXT,
  league_id TEXT,
  score TEXT, 
  line_score TEXT, 
  at_bats REAL, 
  hits REAL, 
  doubles REAL, 
  triples REAL, 
  homeruns REAL, 
  rbi REAL, 
  sacrifice_hits REAL, 
  sacrifice_flies REAL, 
  hit_by_pitch REAL, 
  walks REAL, 
  intentional_walks REAL, 
  strikeouts REAL, 
  stolen_bases REAL, 
  caught_stealing REAL, 
  grounded_into_double REAL, 
  first_catcher_interference REAL, 
  left_on_base REAL, 
  pitchers_used REAL, 
  individual_earned_runs REAL, 
  team_earned_runs REAL, 
  wild_pitches REAL, 
  balks REAL, 
  putouts REAL, 
  assists REAL, 
  errors REAL, 
  passed_balls REAL, 
  double_plays REAL, 
  triple_plays REAL,
  PRIMARY KEY (team_id, game_id),
  FOREIGN KEY (team_id) REFERENCES team(team_id),
  FOREIGN KEY (game_id) REFERENCES game(game_id),
  FOREIGN KEY (league_id) REFERENCES league(league_id)
  );
'''

run_command(team_appearance_table_query)

In [153]:
st = '"v_hits" REAL, "v_doubles" REAL, "v_triples" REAL, "v_homeruns" REAL, "v_rbi" REAL, "v_sacrifice_hits" REAL, "v_sacrifice_flies" REAL, "v_hit_by_pitch" REAL, "v_walks" REAL, "v_intentional_walks" REAL, "v_strikeouts" REAL, "v_stolen_bases" REAL, "v_caught_stealing" REAL, "v_grounded_into_double" REAL, "v_first_catcher_interference" REAL, "v_left_on_base" REAL, "v_pitchers_used" REAL, "v_individual_earned_runs" REAL, "v_team_earned_runs" REAL, "v_wild_pitches" REAL, "v_balks" REAL, "v_putouts" REAL, "v_assists" REAL, "v_errors" REAL, "v_passed_balls" REAL, "v_double_plays" REAL, "v_triple_plays" REAL'
st = st.replace('\"','')
st = st.replace(' REAL','')
print(st)

v_hits, v_doubles, v_triples, v_homeruns, v_rbi, v_sacrifice_hits, v_sacrifice_flies, v_hit_by_pitch, v_walks, v_intentional_walks, v_strikeouts, v_stolen_bases, v_caught_stealing, v_grounded_into_double, v_first_catcher_interference, v_left_on_base, v_pitchers_used, v_individual_earned_runs, v_team_earned_runs, v_wild_pitches, v_balks, v_putouts, v_assists, v_errors, v_passed_balls, v_double_plays, v_triple_plays


In [154]:
st=st.replace('v_','h_')
print(st)

h_hits, h_doubles, h_triples, h_homeruns, h_rbi, h_sacrifice_hits, h_sacrifice_flies, h_hit_by_pitch, h_walks, h_intentional_walks, h_strikeouts, h_stolen_bases, h_caught_stealing, h_grounded_into_double, h_first_catcher_interference, h_left_on_base, h_pitchers_used, h_individual_earned_runs, h_team_earned_runs, h_wild_pitches, h_balks, h_putouts, h_assists, h_errors, h_passed_balls, h_double_plays, h_triple_plays


In [155]:
q = '''
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits, h_doubles, h_triples, h_homeruns, h_rbi, h_sacrifice_hits, h_sacrifice_flies, h_hit_by_pitch, h_walks, h_intentional_walks, h_strikeouts, h_stolen_bases, h_caught_stealing, h_grounded_into_double, h_first_catcher_interference, h_left_on_base, h_pitchers_used, h_individual_earned_runs, h_team_earned_runs, h_wild_pitches, h_balks, h_putouts, h_assists, h_errors, h_passed_balls, h_double_plays, h_triple_plays
    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits, v_doubles, v_triples, v_homeruns, v_rbi, v_sacrifice_hits, v_sacrifice_flies, v_hit_by_pitch, v_walks, v_intentional_walks, v_strikeouts, v_stolen_bases, v_caught_stealing, v_grounded_into_double, v_first_catcher_interference, v_left_on_base, v_pitchers_used, v_individual_earned_runs, v_team_earned_runs, v_wild_pitches, v_balks, v_putouts, v_assists, v_errors, v_passed_balls, v_double_plays, v_triple_plays
    from game_log;
'''

run_command(q)

In [156]:
q='SELECT * FROM team_appearance where league_id="AL" and home =0 LIMIT 10;'
run_query(q)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ANA,ARI200006090,0,AL,1,010000000,31.0,5.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,10.0,1.0,1.0,1.0,0.0,5.0,4.0,3.0,3.0,0.0,0.0,24.0,11.0,2.0,0.0,1.0,0.0
1,ANA,ARI200006100,0,AL,10,033301000,38.0,13.0,3.0,0.0,3.0,10.0,2.0,0.0,0.0,4.0,2.0,6.0,0.0,0.0,0.0,0.0,7.0,4.0,3.0,3.0,0.0,0.0,27.0,9.0,0.0,0.0,1.0,0.0
2,ANA,ARI200006110,0,AL,2,010000100,33.0,8.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,8.0,0.0,0.0,0.0,0.0,5.0,2.0,3.0,3.0,0.0,0.0,24.0,10.0,0.0,0.0,1.0,0.0
3,ANA,ARI200606230,0,AL,8,311000300,34.0,9.0,3.0,0.0,2.0,8.0,0.0,1.0,0.0,7.0,0.0,5.0,0.0,0.0,1.0,0.0,7.0,4.0,2.0,2.0,0.0,0.0,27.0,10.0,0.0,0.0,2.0,0.0
4,ANA,ARI200606240,0,AL,6,0000004000002,50.0,12.0,4.0,1.0,1.0,6.0,1.0,0.0,1.0,3.0,0.0,10.0,0.0,0.0,1.0,0.0,10.0,6.0,4.0,4.0,0.0,0.0,39.0,13.0,1.0,0.0,1.0,0.0
5,ANA,ARI200606250,0,AL,7,110201101,31.0,8.0,1.0,1.0,1.0,5.0,1.0,0.0,0.0,6.0,0.0,5.0,2.0,1.0,1.0,0.0,4.0,3.0,8.0,8.0,0.0,0.0,24.0,10.0,1.0,0.0,1.0,0.0
6,ANA,ARI200906260,0,AL,12,080100030,39.0,11.0,4.0,1.0,1.0,12.0,0.0,0.0,2.0,6.0,0.0,5.0,2.0,0.0,0.0,0.0,8.0,3.0,3.0,3.0,0.0,0.0,27.0,9.0,0.0,0.0,2.0,0.0
7,ANA,ARI200906270,0,AL,2,000001001,31.0,6.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,3.0,0.0,9.0,0.0,0.0,1.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,27.0,10.0,1.0,0.0,1.0,0.0
8,ANA,ARI200906280,0,AL,12,000450012,38.0,11.0,1.0,1.0,3.0,10.0,0.0,0.0,0.0,5.0,0.0,6.0,1.0,2.0,0.0,0.0,4.0,5.0,8.0,8.0,0.0,0.0,27.0,11.0,0.0,0.0,0.0,0.0
9,ANA,ARI201506170,0,AL,2,000011000,36.0,11.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,3.0,0.0,8.0,4.0,3.0,3.0,0.0,0.0,24.0,5.0,0.0,0.0,0.0,0.0


### Person Appearance Table

In [157]:
q = '''
CREATE TABLE IF NOT EXISTS person_appearance(
    appearance_id INTEGER PRIMARY KEY, 
    person_id TEXT, 
    team_id TEXT,
    game_id TEXT,
    appearance_type_id TEXT,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);

'''

run_command(q)

In [158]:
q = '''
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
)

    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE [1b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;

'''

run_command(q)

In [160]:
template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""


for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [161]:
q = 'SELECT * FROM person_appearance LIMIT 10;'
run_query(q)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1,maplb901,None,ALT188404300,UHP
1,2,curte801,ALT,ALT188404300,MM
2,3,murpj104,ALT,ALT188404300,PSP
3,4,hodnc101,SLU,ALT188404300,PSP
4,5,sullt101,SLU,ALT188404300,MM
5,6,hoopm101,None,ALT188405020,UHP
6,7,curte801,ALT,ALT188405020,MM
7,8,learj102,ALT,ALT188405020,PSP
8,9,sullt101,SLU,ALT188405020,MM
9,10,taylb103,SLU,ALT188405020,PSP


## Clean Up

In [ ]:
todel = ['game_log', 'park_codes', 'team_codes', 'person_codes']

for each in todel:
    dropTable()